In [2]:
pip install SpeechRecognition


     --------------------------------------- 32.8/32.8 MB 31.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyttsx3


Note: you may need to restart the kernel to use updated packages.


In [4]:
#Importing libraries
import random
import json
import pickle

import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

import numpy as np
import speech_recognition as sr
import pyttsx3
import time

#Initialization
lemmatizer = WordNetLemmatizer()
intents = json.loads(open("intentsUpdated (1).json").read())

In [6]:
# Loading the vocabulary and intent classes:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
# Loading the pre-trained model:
model = load_model('chatbot_model.h5')

FileNotFoundError: [Errno 2] No such file or directory: 'words.pkl'

In [3]:
# Tokenization and Lemmatization
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word)
                      for word in sentence_words]
 
    return sentence_words
 


In [4]:
 # Converting bag-of-words vector to a NumPy array
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
 
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)
 
 


In [5]:
# Predicting intent of a given sentence using pre-trained model
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
 
    ERROR_THRESHOLD = 0.25
 
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    if not results:
        print("No prediction confidence above threshold.")
        return []
 
    results.sort(key=lambda x: x[1], reverse=True)
 
    return_list = []
 
    for r in results:
        return_list.append({'intent': classes[r[0]],
                            'probability': str(r[1])})
    return return_list
 
 


In [6]:
# Retrieving a response based on the predicted intent
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
 
    result = ''
 
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [7]:
# This function will take the voice input converted
# into string as input and predict and return the result in both
# text as well as voice format.
def calling_the_bot(txt):
    global res
    try:
        predict = predict_class(txt) # Trying to make predictions and get a response
        
        res = get_response(predict, intents)
    
        engine.say("Found it. From our Database we found that" + res) # Speech synthesis and printing the result:
        # engine.say(res)
        engine.runAndWait()
        print("Your Symptom was  : ", txt)
        print("Result found in our Database : ", res)
    except Exception as e: # Exception handling
        print("An error occurred:", str(e))
        engine.say("Unable to find a confident enough prediction.")
        return "An error occurred while processing your request."

In [8]:
# Speech recognition and text-to-speech process to enable voice-based interaction
if __name__ == '__main__':
    print("Bot is Running")
 
    recognizer = sr.Recognizer()
    mic = sr.Microphone()
 
    engine = pyttsx3.init()
    rate = engine.getProperty('rate')
 
    # Increase the rate of the bot according to need,
    # Faster the rate, faster it will speak, vice versa for slower.
 
    engine.setProperty('rate', 175)
 
    # Increase or decrease the bot's volume
    volume = engine.getProperty('volume')
    engine.setProperty('volume', 1.0)
 
    voices = engine.getProperty('voices')
 
    """User Might Skip the following Part till the start of While Loop"""
    engine.say(
        "Hello user, I am NLP Medical Chat bot, your personal Talking Healthcare Chatbot.")
    engine.runAndWait()
 
    engine.say(
        "IF YOU WANT TO CONTINUE WITH MALE VOICE PLEASE\
        SAY MALE. OTHERWISE SAY FEMALE.")
    engine.runAndWait()
 
    # Asking for the MALE or FEMALE voice.
    with mic as source:
        recognizer.adjust_for_ambient_noise(source)
        try:
            audio = recognizer.listen(source)
            choice = recognizer.recognize_google(audio).lower()
        except sr.UnknownValueError:
            engine.say("I didn't catch that. Please repeat.")
            engine.runAndWait()
            choice = "Male"
 
    # If the user says Female then the bot will speak in female voice.
    if choice == "Female".lower():
        engine.setProperty('voice', voices[1].id)
        print("You have chosen to continue with Female Voice")
    else:
        engine.setProperty('voice', voices[0].id)
        print("You have chosen to continue with Male Voice")
 
    """User might skip till HERE"""
 
    while True or final.lower() == 'True':
        with mic as symptom:
            print("Say Your Symptoms. The Bot is Listening")
            engine.say("You may tell me your symptoms now. I am listening")
            engine.runAndWait()
            try:
                recognizer.adjust_for_ambient_noise(symptom)
                symp = recognizer.listen(symptom)
                text = recognizer.recognize_google(symp)
                engine.say("You said {}".format(text))
                engine.runAndWait()
 
                engine.say(
                    "Scanning our database for your symptom. Please wait.")
                engine.runAndWait()
 
                time.sleep(1)
 
                # Calling the function by passing the voice inputted
                # symptoms converted into string
                calling_the_bot(text)
            except sr.UnknownValueError:
                engine.say(
                    "Sorry, Either your symptom is unclear to me or it is\
                    not present in our database. Please Try Again.")
                engine.runAndWait()
 
            engine.say("If you want to continue, please say 'yes', otherwise say 'no'.")
            engine.runAndWait()
 
        with mic as ans:
            recognizer.adjust_for_ambient_noise(ans) 
            try:
                voice = recognizer.listen(ans)
                final = recognizer.recognize_google(voice)
            except sr.UnknownValueError:
                final = "no"
 
        if final.lower() in ['no', 'please exit', 'exit']:
            engine.say("Thank You. Shutting Down now.")
            engine.runAndWait()
            print("Bot has been stopped by the user")
            break

Bot is Running
You have chosen to continue with Female Voice
Say Your Symptoms. The Bot is Listening
1/1 [==============================] - 0s 174ms/step
Your Symptom was  :  sneezing and coughing
Result found in our Database :  According to your symptoms you might have Allergies
Bot has been stopped by the user


In [9]:
# Text-based interface for interacting with the medical chatbot
if __name__ == '__main__':
    print("Chatbot is Running")
    print("Hello! I am NLP Medical Chat bot, your personal Healthcare Chatbot.")
    
    # Loop to handle chat interactions until the user decides to exit
    while True:
        text = input("Please type your symptoms, or type 'exit' to stop: ")
        
        if text.lower() == 'exit':
            print("Thank You. Shutting down now.")
            break

        # Call the function with the user's text input
        calling_the_bot(text)
        
        # Ask if the user wants to continue
        continue_chat = input("Do you want to continue? (yes/no): ")
        if continue_chat.lower() not in ['yes', 'y']:
            print("Exiting. Thank you for using the chatbot.")
            break

Chatbot is Running
Hello! I am NLP Medical Chat bot, your personal Healthcare Chatbot.
Please type your symptoms, or type 'exit' to stop: sneezing and coughing
1/1 [==============================] - 0s 50ms/step
Your Symptom was  :  sneezing and coughing
Result found in our Database :  I suppose you have Allergies
Do you want to continue? (yes/no): no
Exiting. Thank you for using the chatbot.
